In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.7 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [4]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [6]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [7]:
# When using as interactive AI, uncomment this whole section
"""
medical_emergency = input("Hello, what medical emergency are you dealing with?")
zipcode = int(input("What zipcode are you currently living now?"))

medical_request = {
    'Zipcode': zipcode, 
    'Medical Emergency': medical_emergency
}
medical_request_str = str(medical_request)
medical_request_str
"""

# When saving to GitHub, uncomment this whole section 
medical_emergency = 'broken arm'
zipcode = 77035

medical_request = {
    'Zipcode': zipcode, 
    'Medical Emergency': medical_emergency
}
medical_request_str = str(medical_request)
medical_request_str

"{'Zipcode': 77035, 'Medical Emergency': 'broken arm'}"

In [8]:
import typing_extensions as typing

class MedicalRequest(typing.TypedDict): 
    zipcode: int
    medical_emergency: str

response = client.models.generate_content(
    model='gemini-2.0-flash', 
    config=types.GenerateContentConfig(
        temperature=0.1, 
        response_mime_type="application/json", 
        response_schema=MedicalRequest
    ), 
    contents=medical_request_str
)

json_form = response.text

print(json_form)

{
  "zipcode": 77035,
  "medical_emergency": "broken arm"
}


In [9]:
few_shot_prompt = """Parse JSON into a string request and only return output: 

EXAMPLE: 
{
"zipcode": 78664, 
"medical_emergency": "heart attack" 
}
"Patient currently living in zipcode 78664 wants to seek medical attention for heart attack."

EXAMPLE: 
{
"zipcode": 77055, 
"medical_emergency": "broken leg" 
}
"Patient currently living in zipcode 77055 wants to seek medical attention for broken leg."

EXAMPLE: 
{
"zipcode": 78681, 
"medical_emergency": "dog bite" 
}
"Patient currently living in zipcode 78681 wants to seek medical attention for dog bite."
"""

model_config = types.GenerateContentConfig(
    temperature=0.1, 
    top_p=1, 
    max_output_tokens=500
)

response = client.models.generate_content(
    model='gemini-2.0-flash', 
    config=model_config,
    contents=[few_shot_prompt, json_form]
)

str_request = response.text
print(str_request)

"Patient currently living in zipcode 77035 wants to seek medical attention for broken arm."



In [10]:
prompt_config = types.GenerateContentConfig(
    temperature=0.1, 
    top_p=1.0, 
    max_output_tokens=250
)

generate_prompt_prompt = "Generate precise prompt that asks for nearest medical centers for this request: " + str_request

response = client.models.generate_content(
    model='gemini-2.0-flash', 
    config=prompt_config, 
    contents= generate_prompt_prompt
)

prompt = response.text
print(prompt)

Find the nearest medical centers, including urgent care facilities and hospitals, to the patient's location in zip code 77035 that are equipped to diagnose and treat a broken arm. Please provide the name, address, phone number, and estimated distance from the center of zip code 77035 for each location. Prioritize facilities with shorter wait times and positive patient reviews, if available.



In [11]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

response = client.models.generate_content(
    model='gemini-2.0-flash', 
    contents=prompt, 
    config=config_with_search,
)

rc = response.candidates[0]

Markdown(rc.content.parts[0].text)

Okay, I will provide you with a list of medical facilities near the 77035 zip code that can diagnose and treat a broken arm. I will prioritize facilities with shorter wait times and positive patient reviews where possible, and include the name, address, phone number, and estimated distance from the center of 77035.

**Please note:** Wait times can fluctuate significantly and the below information may not reflect real-time availability. Always call ahead to confirm hours, services, and current wait times. Also, distances are approximate and calculated from the center of zip code 77035.

Here's what I've found, combining urgent care centers and hospitals:

**Hospitals**

*   **HCA Houston Healthcare Medical Center**

    *   Address: 1313 Hermann Dr. Houston, TX 77004
    *   Phone: (713) 527 - 5000
    *   Distance: Approximately 3.7 miles
    *   Notes: This hospital offers comprehensive orthopedic care for a wide range of injuries and conditions, including broken bones. They provide advanced pain management, rehabilitation services, and minimally invasive surgical procedures. HCA Houston Healthcare facilities strive to minimize ER wait times.
*   **Memorial Hermann-Texas Medical Center**

    *   Address: 6411 Fannin St, Houston, TX 77030
    *   Phone: (713) 704-4000
    *   Distance: Approximately 4.5 miles
    *   Notes: This is a large teaching hospital with a wide range of medical experts, including orthopedic specialists.
*   **Ben Taub Hospital**

    *   Address: Not specified in search results, but located in Houston, TX.
    *   Notes: This hospital has been recognized for awards in clinical quality and patient safety.

**Urgent Care Centers**

*   **Memorial Primary and Urgent Care**

    *   Address: 14629 Memorial Drive, Houston, TX 77079
    *   Phone: (281) 589-8500
    *   Notes: This facility diagnoses and treats fractures, offering X-ray and other diagnostic services. Patient reviews highlight the professionalism and efficiency of the staff.
*   **Lifeline Urgent Care**

    *   Address: (Location in Houston, TX - specific address needed for distance)
    *   Phone: 281-771-1144
    *   Notes: Lifeline Urgent Care provides walk-in bone fracture treatment with X-ray capabilities and splinting. They can also refer to orthopedic specialists. They advertise lower rates and shorter wait times compared to emergency rooms.
*   **AFC Urgent Care Memorial City TX**

    *   Address: (Specific address needed for distance from 77035)
    *   Phone: (346) 385-5989
    *   Notes: They offer urgent care for patients of all ages and have X-Ray services on-site. They advertise shorter wait times than emergency rooms.
*   **Next Level Urgent Care**

    *   Address: (Multiple locations, check website for the nearest to 77035)
    *   Notes: Known for prompt service and friendly staff, Next Level Urgent Care offers immediate care for non-life-threatening injuries.
*   **Insight Urgent Care**

    *   Address: (Multiple locations, check website for the nearest to 77035)
    *   Notes: Patients have reported quick service and expert treatment at Insight Urgent Care. They offer X-rays for minor injuries.

**Important Considerations:**

*   **Severity of the Break:** For severe breaks, especially compound fractures (where the bone breaks the skin), an emergency room is the most appropriate choice. They have the resources for immediate surgery and to prevent infection.
*   **Wait Times:** Call the facility to ask about current wait times. Urgent care centers generally have shorter wait times than hospital ERs, but this can vary. Some hospitals and urgent care centers may offer online check-in or wait time information.
*   **Insurance:** Verify that the facility accepts your insurance.
*   **Pediatric Care:** If the patient is a child, consider facilities specializing in pediatric orthopedics.
*   **Orthopedic Specialists:** Confirm that the facility has orthopedic specialists available or can provide a referral if needed.

I hope this information is helpful. Remember to call the facilities directly to confirm details and make the best choice for the patient's specific needs.
